In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras


MODEL_DIR = f"models/assignment_transformer_HLF"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)
plt.rcParams.update({"font.size": 14})

data_config = DataProcessor.load_from_npz(
    data_configs["data_path"]["nominal"], max_events=2_000_000
)
X_train, y_train, _, _ = DataProcessor.split_even_odd()
del DataProcessor

2025-12-02 10:09:47.847750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764666587.871428  519971 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764666587.879270  519971 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764666587.897459  519971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764666587.897479  519971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764666587.897482  519971 computation_placer.cc:177] computation placer alr

In [ ]:
TransformerMatcher = Models.FeatureConcatTransformer(data_config, name=r"Transformer")

TransformerMatcher.build_model(
    hidden_dim=64,
    num_layers=6,
    num_heads=8,
    dropout_rate=0.1,
    compute_HLF=True,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)

FeatureConcatTransformer is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1764666697.414300  519971 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.


I0000 00:00:1764666699.539109  522822 service.cc:152] XLA service 0x7f4d78003680 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764666699.539138  522822 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-12-02 10:11:39.585415: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1764666699.669032  522822 cuda_dnn.cc:529] Loaded cuDNN version 91500


147/472 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

...

472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Adapted normalization layer:  hlf_input_normalization
 49/472 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step    

2025-12-02 10:11:41.795868: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert


472/472 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


2025-12-02 10:11:42.443075: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert


Adapted normalization layer:  met_input_normalization
472/472 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Adapted normalization layer:  lep_input_normalization
472/472 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Adapted normalization layer:  jet_input_normalization


In [3]:
TransformerMatcher.train_model(
    epochs=100,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1,
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=15,
            restore_best_weights=True,
            verbose=1,
        ),
    ],
)

Epoch 1/100


2025-12-02 10:12:03.670582: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


778/781 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3337 - loss: 0.3588

2025-12-02 10:12:35.105048: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3340 - loss: 0.3587

2025-12-02 10:12:52.644046: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-02 10:12:54.949392: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


781/781 ━━━━━━━━━━━━━━━━━━━━ 70s 45ms/step - accuracy: 0.3341 - loss: 0.3587 - val_accuracy: 0.5822 - val_loss: 0.1029 - learning_rate: 1.0000e-04
Epoch 2/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.5398 - loss: 0.2271 - val_accuracy: 0.6717 - val_loss: 0.0899 - learning_rate: 1.0000e-04
Epoch 3/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.5785 - loss: 0.2947 - val_accuracy: 0.6896 - val_loss: 0.0866 - learning_rate: 1.0000e-04
Epoch 4/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.5896 - loss: 0.3175 - val_accuracy: 0.6930 - val_loss: 0.0843 - learning_rate: 1.0000e-04
Epoch 5/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.5930 - loss: 0.3470 - val_accuracy: 0.7093 - val_loss: 0.0814 - learning_rate: 1.0000e-04
Epoch 6/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.5938 - loss: 0.3699 - val_accuracy: 0.7175 - val_loss: 0.0796 - learning_rate: 1.0000e-04
Epoch 7/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - a

In [4]:
TransformerMatcher.save_model(f"{MODEL_DIR}/model.keras")

Model saved to models/assignment_transformer_HLF//model.keras
